# Red-black trees

* Red-black trees are a type of self-balancing binary search tree. They have $O(\log{n})$ read times, making them highly efficient.
* In addition to the properties of a binary search tree red-black trees have the following invariants:
  * Each node is either red or black.
  * The root is black.
  * All leaves are `null` and black.
  * If a node is red, then both its children are black.
  * Every path from a given node to any of its descendant `null` nodes contains the same number of black nodes.
* Red–black trees offer worst-case guarantees for insertion time, deletion time, and search time.
* Read operations are the same as in binary search trees.
* Insert and delete are much more compliated, but still $O(\log{n})$.
* Here's the tutorial I'm following: http://eternallyconfuzzled.com/tuts/datastructures/jsw_tut_rbtree.aspx.


* A

In [1]:
class Node:
    def __init__(color, left, right):
        self.color = color
        self.left = left
        self.right = right
        

class Tree:
    def __init__(nodes=None):
        self.nodes = nodes